# 1- Importing Libs.

In [1]:
!pip install evaluate rouge-score
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, pipeline
import torch
from tqdm import tqdm
import evaluate
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import os

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=08372434f77f432ed26b5126996eb4e8b92a4b533d335c2e939a48e002d0444a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# 2. Preparing environment

In [54]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf_token")

In [55]:

os.environ['HF_TOKEN']=secret_value_0

login(token=os.getenv('HF_TOKEN'))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
model_ckpt = "google/pegasus-cnn_dailymail"
device = 'cuda' if torch.cuda.is_available() else "cpu"

# 3. loading and taking a look at the dataset

In [3]:
dataset = load_dataset("knkarthick/dialogsum")

README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [7]:
print(dataset['train']['dialogue'][0], dataset['train']['summary'][0], sep='\n\n')

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.

Mr. Smith's getting a check-up, and Doctor Hawkins advises him to h

# 4. Evaluating the model before fine tuning

In [8]:
metric = evaluate.load("rouge")

def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries(model, 
                       tokenizer, 
                       dataset, 
                       batch_size, 
                       metric, 
                       col_name='dialogue', 
                       col_summary='summary'):
    article_batches = list(chunks(dataset[col_name], batch_size))
    summary_batches = list(chunks(dataset[col_summary], batch_size))

    for article_batch, summary_batch in tqdm(zip(article_batches, summary_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, 
                           truncation=True, return_tensors='pt', padding='max_length')
        summaries = model.generate(input_ids=inputs['input_ids'].to(device), 
                                   attention_mask=inputs['attention_mask'].to(device), 
                                   max_length=128, length_penalty=0.8, num_beams=5)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                              clean_up_tokenization_spaces=True) 
                             for s in summaries]
        decoded_summaries = [d.replace('<n>', " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=summary_batch)

    score = metric.compute()
    return score

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

score = evaluate_summaries(model, tokenizer, dataset['validation'][:100], 4, metric)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 25/25 [01:34<00:00,  3.76s/it]


In [11]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=["PEGASUS"])

,rouge1,rouge2,rougeL,rougeLsum
PEGASUS,0.261658,0.067741,0.201517,0.20154


# 5. Fine Tuning PEGASUS

In [13]:
def tokenize(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], truncation=True, max_length=128)

    return {
        "input_ids": input_encodings['input_ids'], 
        "attention_mask": input_encodings['attention_mask'], 
        "labels": target_encodings['input_ids']
    }


train_dataset = dataset['train'].map(tokenize, batched=True) 
val_dataset = dataset['validation'].map(tokenize, batched=True) 
test_dataset = dataset['test'].map(tokenize, batched=True) 

columns = ['input_ids', 'labels', 'attention_mask']
train_dataset.set_format(type='torch', columns=columns)
val_dataset.set_format(type="torch", columns=columns)
test_dataset.set_format(type="torch", columns=columns)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [15]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [53]:
training_arguments = Seq2SeqTrainingArguments(
    output_dir='pegasus-dialogue',  
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,
    weight_decay=0.01, 
    logging_steps=50, 
    push_to_hub=True, 
    eval_strategy='steps', 
    eval_steps=1000, 
    save_steps=1e6, 
    gradient_accumulation_steps=16,
    predict_with_generate=True,
    report_to=[]
)

In [56]:
trainer = Seq2SeqTrainer(model=model, 
                         tokenizer=tokenizer, 
                         args=training_arguemnts, 
                         data_collator=seq2seq_data_collator, 
                         train_dataset=train_dataset.select(range(6000)), 
                         eval_dataset=val_dataset
                        )

/tmp/ipykernel_23/963016224.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(model=model,


In [57]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=374, training_loss=1.6713089904683158, metrics={'train_runtime': 4022.8018, 'train_samples_per_second': 2.983, 'train_steps_per_second': 0.093, 'total_flos': 8679313386455040.0, 'train_loss': 1.6713089904683158, 'epoch': 1.9946666666666668})

# 6. Evaluating on the validation set

In [58]:
score = evaluate_summaries(trainer.model, tokenizer, dataset['test'][:100], 4, metric)
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=["fine-tuned-PEGASUS"])

100%|██████████| 25/25 [01:10<00:00,  2.83s/it]


,rouge1,rouge2,rougeL,rougeLsum
fine-tuned-PEGASUS,0.366658,0.119804,0.289407,0.290264


In [59]:
trainer.push_to_hub('pegasus-dialogsum')

CommitInfo(commit_url='https://huggingface.co/Ahmed167/pegasus-dialogue/commit/755c1e623fb3a405b38b5daf9d3d826e7322e250', commit_message='pegasus-dialogsum', commit_description='', oid='755c1e623fb3a405b38b5daf9d3d826e7322e250', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ahmed167/pegasus-dialogue', endpoint='https://huggingface.co', repo_type='model', repo_id='Ahmed167/pegasus-dialogue'), pr_revision=None, pr_num=None)

## A sample of the test set

In [62]:
pipe = pipeline('summarization', model="Ahmed167/pegasus-dialogue")
sample = train_dataset.select(range(1))
dialogue = sample['dialogue']
reference = sample['summary']

print(f"Dialogue:\n{dialogue[0]}")
print("*"*20)
print(f"model summary:\n{pipe(dialogue, length_penalty=0.8, num_beams=8, max_length=128)[0]['summary_text']}")
print("*"*20)
print(f"reference:\n{reference[0]}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Dialogue:
#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.
********************
model summary:
Mr. Smith visits Docto